In [5]:
import pandas as pd

headers = ['user_id', 'game', 'behavior', 'hours_played', 'other_columns']
# Load the dataset
data = pd.read_csv('steam-200k.csv', header=None, names=headers)

# Data cleaning
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)

# Filter games and users with sufficient interactions
data = data[data['hours_played'] > 0]

# Split into training and testing sets
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2)

In [6]:
import networkx as nx

# Create a bipartite graph
G = nx.Graph()

# Add nodes and edges
for row in train_data.itertuples():
    G.add_node(row.user_id, bipartite=0)
    G.add_node(row.game, bipartite=1)
    G.add_edge(row.user_id, row.game, weight=row.hours_played)

In [27]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

model = Word2Vec.load("./models/node2vec_model_walk_1000_length_60")

# Get embeddings for all nodes
embeddings = model.wv

def recommend(user_id, num_recommendations=5):
    user_id_str = str(user_id)
    if user_id_str not in embeddings:
        print(f"User ID {user_id} was not found in embeddings")
        return []
    user_embedding = embeddings[str(user_id)]

    # Filter item nodes (those with bipartite=1)
    item_nodes = [node for node, data in G.nodes(data=True) if data.get('bipartite') == 1]
    
    # Compute similarity between user embedding and all item embeddings
    item_embeddings = [embeddings[str(item)] for item in item_nodes if str(item) in embeddings]
    similarities = cosine_similarity([user_embedding], item_embeddings).flatten()
    
    # Get top N most similar items
    top_indices = similarities.argsort()[-num_recommendations:][::-1]
    recommended_items = [item_nodes[i] for i in top_indices]
    
    return recommended_items

sample_user_id = 23717586
# Example recommendation
recommendations = recommend(user_id=sample_user_id, num_recommendations=5)
print(recommendations)

["Don't Starve Reign of Giants", 'Battlefield Bad Company 2', "Hero Siege - The Karp of Doom (Digital Collector's Edition)", 'Max Payne 3', 'Black Mirror II']
